The Long Short – Term Memory (LSTM) is a RNN architecture that developed to overcome the vanishing gradient problem.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file
data = pd.read_csv("filled.csv")
data = data.set_index(pd.to_datetime(data['date']))
data.describe()

In [ ]:
data = data.sort_index()
data = data.reset_index(drop=True)
dates = data["date"]
data = data.drop(["date"], axis=1)

In [ ]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
inspect_dataframe(data, data.columns)

In [ ]:
threshold = 6 # theshold a little high to retain some outliers
z_scores = np.abs((data - data.mean()) / data.std())
outliers = (z_scores > threshold).any(axis=1)
for column in data.columns:
    column_median = data[column].median()
    data.loc[outliers, column] = column_median

In [ ]:
# Normalize the data, this is a rudimentary MinMaxScaler
max_df = data.max()
min_df = data.min()

data_norm = (data - min_df)/(max_df - min_df)
data_norm = pd.DataFrame(data_norm, columns=data.columns)

In [ ]:
def build_sequences(df, target_labels=['BC'], window=200, stride=200):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.drop(['BC'], axis=1).copy().values
    temp_label = df[target_labels].copy().values
    # padding_len = len(df) % window
    
    # if padding_len != 0:
    #     # Compute padding length
    #     padding_len = window - len(df) % window
    #     padding = np.zeros((padding_len, temp_df.shape[1]), dtype='float32')
    #     temp_df = np.concatenate((padding, temp_df))
    #     padding = np.zeros((padding_len,1), dtype='float32')
    #     #padding = np.zeros((padding_len, temp_label.shape[1]), dtype='float32')
    #     temp_label = np.concatenate((padding, temp_label))
    #     assert len(temp_df) % window == 0

    # Build sequences and labels
    for i in range(0, len(temp_df) - window + 1, stride):
        dataset.append(temp_df[i:i + window])
        labels.append(temp_label[i:i + window])

    return np.array(dataset), np.array(labels)


In [ ]:
test_size = 24*10
val_size = 24*15
window=24*8
stride=2

train = data_norm.iloc[val_size:-test_size]
val= data_norm.iloc[0:val_size]
test = data_norm.iloc[-test_size:]


print(train.shape, test.shape)
# data_norm.describe()
X_train, y_train = build_sequences(train, window=window, stride=stride)
X_val, y_val = build_sequences(val, window=window, stride=stride)
X_test, y_test = build_sequences(test, window=window, stride=stride)
X_train.shape, y_train.shape,  X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
import tensorflow as tf
import os
import random
import seaborn as sns
import matplotlib as mpl
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
tfk = tf.keras
tfkl = tf.keras.layers
#tf.config.set_visible_devices([], 'GPU') #disables GPU
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[-1])))
model.add(LSTM(20, return_sequences=True))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mse', optimizer='rmsprop')

In [ ]:
batch_size = 128
epochs = 5000

In [ ]:
inspect_dataframe(data_norm, data.columns)

In [ ]:
history = model.fit(
    x = X_train,
    y = y_train, 
    batch_size = batch_size,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
                            ],
    epochs = epochs
).history

In [ ]:
y_pred=model.predict(X_test)

In [23]:
y_test.shape, y_pred.shape

((25, 192, 1), (25, 192, 1))

In [29]:
from sklearn.metrics import mean_absolute_error


y_pred
mae = mean_absolute_error(y_test.reshape(-1, 1), y_pred.reshape(-1, 1))
print(mae)

0.047264987665346335


In [ ]:
# y_pred = model.predict(X_test)
# y_val_actual = scaler.inverse_transform(np.hstack((y_val.reshape(-1, 1), X_test[:, -1])))[:, 0]
# y_pred_actual = scaler.inverse_transform(np.hstack((y_pred.reshape(-1, 1), X_test[:, -1])))[:, 0]

# print(y_val_actual)
# print(y_pred_actual)

# mae = mean_absolute_error(y_val_actual, y_pred_actual)
# mse = mean_squared_error(y_val_actual, y_pred_actual)
# rmse = np.sqrt(mse)

# print('MAE:', mae)
# print('MSE:', mse)
# print('RMSE:', rmse)

# # Visualize the model's predictions
# plt.plot(y_val_actual, label='Actual')
# plt.plot(y_pred_actual, label='Predicted')
# plt.legend()
# plt.show()